In [10]:
# [Cell 1] - Import statements
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from tabulate import tabulate
from typing import Dict, List, Any
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization

In [11]:
# [Cell 2] - Set global constants and configurations [config.py]
RANDOM_SEED = 42

EXPERIMENT_BASE_DIR = 'exp-04'
MODEL_DIR = os.path.join(EXPERIMENT_BASE_DIR, "pretext_models")
DATA_DIR = os.path.join(EXPERIMENT_BASE_DIR, "processed_data")
RESULTS_DIR = os.path.join(EXPERIMENT_BASE_DIR, "results")

# Create required directories
for dir_path in [MODEL_DIR, DATA_DIR, RESULTS_DIR]:
    os.makedirs(dir_path, exist_ok=True)

experiment_parameters = {
    # Parameters
    'mode': 'all',  # choices: 'process', 'pretext', 'downstream', 'all'
    'split_index': 0,
    'architecture': 'cnn',  # options: specific architecture like 'cnn' or 'all'
    'classifier': 'all',  # or specific: 'random_forest', 'svm', 'gradient_boosting', 'xgboost'
    # Data directory
    'data_dir': '/mnt/c/Users/Siam/OneDrive - Tuskegee University/ai-arni-nsf/SAMPLE_dataset_public/png_images/qpm/real'
}

# Configuration dictionary
hyperparameters = {
    # Data loading and splitting
    'img_size': (128, 128),
    'color_mode': 'grayscale',
    'test_split_size': 0.2,
    'test_split_index': 0,

    # Model architecture
    'input_shape': (128, 128, 1),
    'num_classes': 4,
    'fine_tune': True,

    # CNN architecture
    'cnn_filters': [32, 64, 128, 256],
    'cnn_kernel_size': (3, 3),
    'cnn_pool_size': (2, 2),
    'cnn_dense_units': 512,
    'cnn_dropout_rate': 0.5,
    'cnn_loss_function': 'sparse_categorical_crossentropy',
    'cnn_activation_function': 'relu',
    'cnn_padding': 'same',

    # Other Pretext Architectures
    'pretained_model_weights': 'imagenet',

    # Training configuration
    'batch_size': 32,
    'epochs': 10,
    'validation_split': 0.25,
    'learning_rate': 0.001,
    'early_stopping_patience': 3,
    'lr_reduction_patience': 3,
    'lr_reduction_factor': 0.5,

    # Downstream task
    'feature_extraction_layer': -2,
    'rf_n_estimators': 100,
    'svm_kernel': 'linear',
    'gb_n_estimators': 100,
    'xgb_n_estimators': 100,
    'cv_splits': 5,
}

architecture_list = [
    'cnn',
    'resnet50',
    'resnet101',
    'resnet152',
    'efficientnetb0',
    'vgg16',
    'vgg19',
    'inceptionv3',
    'unet'
]

classifier_list = [
    'random_forest', 
    'svm', 
    'gradient_boosting', 
    'xgboost'
]

def display_configurations():
    print("\n===== Experiment Configurations =====\n")
    
    print(">>> Data Loading and Splitting <<<")
    for key in ['img_size', 'color_mode', 'test_split_size', 'test_split_index']:
        print(f"{key}: {hyperparameters[key]}")
    
    print("\n>>> Model Architecture <<<")
    for key in ['input_shape', 'num_classes', 'fine_tune']:
        print(f"{key}: {hyperparameters[key]}")
    
    print("\n>>> CNN Architecture <<<")
    for key in ['cnn_filters', 'cnn_kernel_size', 'cnn_pool_size', 'cnn_dense_units', 'cnn_dropout_rate', 'cnn_loss_function', 'cnn_activation_function', 'cnn_padding']:
        print(f"{key}: {hyperparameters[key]}")

    print("\n>>> Other Pretext Architectures <<<")
    for key in ['pretained_model_weights']:
        print(f"{key}: {hyperparameters[key]}")
    
    print("\n>>> Training Configuration <<<")
    for key in ['batch_size', 'epochs', 'validation_split', 'learning_rate', 'early_stopping_patience', 'lr_reduction_patience', 'lr_reduction_factor']:
        print(f"{key}: {hyperparameters[key]}")
    
    print("\n>>> Downstream Task Configuration <<<")
    for key in ['feature_extraction_layer', 'rf_n_estimators', 'svm_kernel', 'gb_n_estimators', 'xgb_n_estimators', 'cv_splits']:
        print(f"{key}: {hyperparameters[key]}")
    
    print("\n>>> Architectures <<<")
    print(", ".join(architecture_list))
    
    print("\n>>> Classifiers <<<")
    print(", ".join(classifier_list))
    
    print("\n>>> Experiment Parameters <<<")
    for key, value in experiment_parameters.items():
        print(f"{key}: {value}")
    
    print("\n=====================================\n")

In [12]:
# [Cell 3] - Utility functions [seeds.py/ResultsTracker.py]
def set_all_seeds(seed=42):
    """Set all seeds to make results reproducible"""
    tf.keras.utils.set_random_seed(seed)
    tf.config.experimental.enable_op_determinism()
    np.random.seed(seed)
    random.seed(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    # Force TensorFlow to use single thread
    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

class ResultsTracker:
    def __init__(self):
        self.results = []

    def add_result(self, data_dir: str, architecture: str, classifier: str, 
                   cv_metrics: Dict[str, List[float]], test_metrics: Dict[str, Any]):
        result = {
            'Dataset': f"{data_dir.split('/')[-2]}_{data_dir.split('/')[-1]}",
            'Architecture': architecture,
            'Classifier': classifier,
            'CV_Accuracy': f"{np.mean(cv_metrics['accuracies'])*100:.1f}±{np.std(cv_metrics['accuracies'])*100:.1f}",
            'CV_Precision': f"{np.mean(cv_metrics['precisions'])*100:.1f}±{np.std(cv_metrics['precisions'])*100:.1f}",
            'CV_Recall': f"{np.mean(cv_metrics['recalls'])*100:.1f}±{np.std(cv_metrics['recalls'])*100:.1f}",
            'CV_F1': f"{np.mean(cv_metrics['f1_scores'])*100:.1f}±{np.std(cv_metrics['f1_scores'])*100:.1f}",
            'Test_Accuracy': f"{test_metrics['accuracy']*100:.1f}",
            'Test_Precision': f"{test_metrics['precision']*100:.1f}",
            'Test_Recall': f"{test_metrics['recall']*100:.1f}",
            'Test_F1': f"{test_metrics['f1']*100:.1f}",
            'Test_Confusion_Matrix': f"{test_metrics['confusion_matrix']}"
        }
        self.results.append(result)

    # def display_results(self):
    #     if not self.results:
    #         print("> No results to display")
    #         return

    #     df = pd.DataFrame(self.results)
    #     grouped = df.groupby(['Dataset', 'Architecture'])

    #     for (dataset, arch), group in grouped:
    #         print(f"\n=== Results: Dataset: <{dataset}> | Pretext Model: <{arch}> ===")
            
    #         display_cols = [
    #             'Classifier',
    #             'Test_Accuracy', 'Test_Precision', 'Test_Recall', 'Test_F1',
    #             'CV_Accuracy', 'CV_Precision', 'CV_Recall', 'CV_F1'
    #         ]
            
    #         display_df = group[display_cols].copy()
    #         print(tabulate(display_df, headers='keys', tablefmt='grid', showindex=False))
            
    #         display_cols = ['Classifier', 'Test_Confusion_Matrix']
    #         display_df = group[display_cols].copy()
    #         print(tabulate(display_df, headers='keys', tablefmt='grid', showindex=False))

    def display_results(self):
        if not self.results:
            print("> No results to display")
            return

        df = pd.DataFrame(self.results)
        grouped = df.groupby(['Dataset', 'Architecture'])

        for (dataset, arch), group in grouped:
            print(f"\n=== Results: Dataset: <{dataset}> | Pretext Model: <{arch}> ===")

            # Display test and cross-validation metrics
            display_cols = [
                'Classifier',
                'Test_Accuracy', 'Test_Precision', 'Test_Recall', 'Test_F1',
                'CV_Accuracy', 'CV_Precision', 'CV_Recall', 'CV_F1'
            ]
            
            display_df = group[display_cols].copy()
            print("\nMetrics Summary:")
            print(tabulate(display_df, headers='keys', tablefmt='grid', showindex=False))

            # Separate classifiers and confusion matrices
            classifier_names = group['Classifier'].tolist()
            confusion_matrices = group['Test_Confusion_Matrix'].tolist()

            # Build display format for classifiers and confusion matrices
            display_confusion_matrix = pd.DataFrame([confusion_matrices], columns=classifier_names, index=["Confusion Matrix"])
            display_classifiers = pd.DataFrame([classifier_names], columns=classifier_names, index=["Classifier"])
            combined_df = pd.concat([display_classifiers, display_confusion_matrix])
            print(tabulate(combined_df, headers='keys', tablefmt='grid', showindex=True))

In [13]:
# [Cell 4] - Data Processing Functions [data_processor.py]
def load_and_split_data(data_dir, split_index=0):
    images_by_class = {}
    labels_by_class = {}

    class_folders = sorted(os.listdir(data_dir))
    for class_index, class_folder in enumerate(class_folders):
        class_path = os.path.join(data_dir, class_folder)
        if os.path.isdir(class_path):
            img_files = sorted(os.listdir(class_path))
            
            images_by_class[class_index] = []
            labels_by_class[class_index] = []
            
            for img_file in img_files:
                img_path = os.path.join(class_path, img_file)
                image = load_img(
                    img_path,
                    target_size=hyperparameters['img_size'],
                    color_mode=hyperparameters['color_mode']
                )
                image = img_to_array(image) / 255.0
                images_by_class[class_index].append(image)
                labels_by_class[class_index].append(class_index)

    train_images, train_labels = [], []
    test_images, test_labels = [], []

    print("\nData Distribution:")
    print(f"{'Class':^10} {'Total':^10} {'Train':^10} {'Test':^10}")
    print("-" * 40)
    
    test_size = hyperparameters['test_split_size']
    for class_idx in sorted(images_by_class.keys()):
        X = np.array(images_by_class[class_idx])
        y = np.array(labels_by_class[class_idx])
        
        total_samples = len(X)
        chunk_size = int(total_samples * test_size)
        start_idx = split_index * chunk_size
        end_idx = start_idx + chunk_size
        
        test_mask = np.zeros(total_samples, dtype=bool)
        test_mask[start_idx:end_idx] = True
        
        test_images.extend(X[test_mask])
        test_labels.extend(y[test_mask])
        train_images.extend(X[~test_mask])
        train_labels.extend(y[~test_mask])

        print(f"{class_idx:^10} {total_samples:^10} {sum(~test_mask):^10} {sum(test_mask):^10}")

    return np.array(train_images), np.array(train_labels), np.array(test_images), np.array(test_labels)

def save_processed_data(data_dir, split_index):
    base_name = f"{data_dir.split('/')[-2]}_{data_dir.split('/')[-1]}_split_{split_index}"
    save_path = os.path.join(DATA_DIR, base_name)

    if os.path.exists(f"{save_path}_train.pkl"):
        print(f"> Data already saved in {save_path}_train.pkl")
        return
    
    x_train, y_train, x_test, y_test = load_and_split_data(data_dir, split_index)

    with open(f"{save_path}_train.pkl", 'wb') as f:
        pickle.dump((x_train, y_train), f)
    with open(f"{save_path}_test.pkl", 'wb') as f:
        pickle.dump((x_test, y_test), f)
    
    print(f"> Saved processed data to {save_path}")
    return save_path

def load_processed_data(base_path):
    with open(f"{base_path}_train.pkl", 'rb') as f:
        x_train, y_train = pickle.load(f)
    with open(f"{base_path}_test.pkl", 'rb') as f:
        x_test, y_test = pickle.load(f)

    test_size = hyperparameters['test_split_size']
    print("\nSummary of Loaded Data:")
    print(f"[DATAINFO] Total images: {len(x_train) + len(x_test)}")
    print(f"[DATAINFO] Training set: {len(x_train)} images ({(1-test_size)*100:.0f}%)")
    print(f"[DATAINFO] Test set: {len(x_test)} images ({test_size*100:.0f}%)")
    print(f"[DATAINFO] Image shape: {x_train[0].shape}")
    print()

    return x_train, y_train, x_test, y_test

In [14]:
# [Cell 6] - Model Building Functions
def build_custom_cnn_model(input_shape=(128, 128, 1), num_classes=4, architecture_name='cnn', fine_tune=True):
    inputs = tf.keras.Input(shape=input_shape)
    
    if architecture_name == 'cnn':
        # Use the values from the hyperparameters dictionary
        x = layers.Conv2D(hyperparameters['cnn_filters'][0], hyperparameters['cnn_kernel_size'], 
                          activation=hyperparameters['cnn_activation_function'], padding=hyperparameters['cnn_padding'])(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(hyperparameters['cnn_pool_size'])(x)
        
        x = layers.Conv2D(hyperparameters['cnn_filters'][1], hyperparameters['cnn_kernel_size'], 
                          activation=hyperparameters['cnn_activation_function'], padding=hyperparameters['cnn_padding'])(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(hyperparameters['cnn_pool_size'])(x)
        
        x = layers.Conv2D(hyperparameters['cnn_filters'][2], hyperparameters['cnn_kernel_size'], 
                          activation=hyperparameters['cnn_activation_function'], padding=hyperparameters['cnn_padding'])(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(hyperparameters['cnn_pool_size'])(x)
        
        x = layers.Conv2D(hyperparameters['cnn_filters'][3], hyperparameters['cnn_kernel_size'], 
                          activation=hyperparameters['cnn_activation_function'], padding=hyperparameters['cnn_padding'])(x)
        x = layers.BatchNormalization()(x)
        x = layers.GlobalAveragePooling2D()(x)

        x = layers.Dense(hyperparameters['cnn_dense_units'], activation=hyperparameters['cnn_activation_function'])(x)
        x = layers.Dropout(hyperparameters['cnn_dropout_rate'])(x)
    else:
        # Upsample input to the required size if needed (InceptionV3 requires minimum 75x75)
        if architecture_name == 'inceptionv3' and (input_shape[0] < 75 or input_shape[1] < 75):
            required_size = (75, 75)  # InceptionV3 minimum size
        else:
            required_size = (input_shape[0], input_shape[1])  # Default size for other models
            
        x = layers.Resizing(required_size[0], required_size[1])(inputs)  # Resize input to required size

        # Convert grayscale (1-channel) to 3-channel RGB for pretrained models
        x = layers.Conv2D(3, (1, 1))(x)
        
        # Pretrained model selection logic
        # ResNet Variants
        if architecture_name == 'resnet50':
            from tensorflow.keras.applications import ResNet50
            base_model = ResNet50(weights=hyperparameters['pretained_model_weights'], include_top=False, input_shape=(required_size[0], required_size[1], 3))
        elif architecture_name == 'resnet101':
            from tensorflow.keras.applications import ResNet101
            base_model = ResNet101(hyperparameters['pretained_model_weights'], include_top=False, input_shape=(required_size[0], required_size[1], 3))
        elif architecture_name == 'resnet152':
            from tensorflow.keras.applications import ResNet152
            base_model = ResNet152(weights=hyperparameters['pretained_model_weights'], include_top=False, input_shape=(required_size[0], required_size[1], 3))
        
        # EfficientNetB0
        elif architecture_name == 'efficientnetb0':
            from tensorflow.keras.applications import EfficientNetB0
            base_model = EfficientNetB0(weights=hyperparameters['pretained_model_weights'], include_top=False, input_shape=(required_size[0], required_size[1], 3))

        # VGGNet Variants
        elif architecture_name == 'vgg16':
            from tensorflow.keras.applications import VGG16
            base_model = VGG16(weights=hyperparameters['pretained_model_weights'], include_top=False, input_shape=(required_size[0], required_size[1], 3))
        elif architecture_name == 'vgg19':
            from tensorflow.keras.applications import VGG19
            base_model = VGG19(weights=hyperparameters['pretained_model_weights'], include_top=False, input_shape=(required_size[0], required_size[1], 3))

        # InceptionV3
        elif architecture_name == 'inceptionv3':
            from tensorflow.keras.applications import InceptionV3
            base_model = InceptionV3(weights=hyperparameters['pretained_model_weights'], include_top=False, input_shape=(required_size[0], required_size[1], 3))

        # U-Net (not from Keras applications, custom U-Net function)
        elif architecture_name == 'unet':
            base_model = build_unet_model(input_shape=(required_size[0], required_size[1], 3))  # Custom function to build U-Net model

        # Set base model to non-trainable if fine-tuning is disabled
        if not fine_tune:
            base_model.trainable = False
        else:
            base_model.trainable = True
        
        # Apply base model to input
        x = base_model(x)
        x = GlobalAveragePooling2D()(x)
    
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    
    # Compile the model using learning rate and loss from the hyperparameters
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hyperparameters['learning_rate']),
        loss= hyperparameters['cnn_loss_function'],
        metrics=['accuracy']
    )

    model.summary()
    
    return model

In [15]:
# [Cell 7] - Pretext Task Functions
def augment_image(image, rotation_angle):
    if rotation_angle == 90:
        image = tf.image.rot90(image)
    elif rotation_angle == 180:
        image = tf.image.rot90(image, k=2)
    elif rotation_angle == 270:
        image = tf.image.rot90(image, k=3)
    
    label = rotation_angle // 90
    return image, label

def preprocess_data(images):
    augmented_images = []
    labels = []
    for image in images:
        for rotation_angle in [0, 90, 180, 270]:
            aug_image, label = augment_image(image, rotation_angle)
            augmented_images.append(aug_image)
            labels.append(label)
    print(f'> {len(augmented_images)} augmented images generated each of shape {augmented_images[0].shape} with {len(labels)} labels')
    return np.array(augmented_images), np.array(labels)

def save_model(model, architecture_name, data_path, split_index):
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_name = f"pretext_model_{os.path.basename(data_path)}_{architecture_name}_split_{split_index}.h5"
        model_path = os.path.join(MODEL_DIR, model_name)
        
        try:
            model.save(model_path, save_format='h5')
        except Exception as h5_error:
            print(f"> H5 saving failed, trying SavedModel format: {h5_error}")
            model_path = model_path.replace('.h5', '')
            model.save(model_path, save_format='tf')
            
        print(f"> Model successfully saved at: {model_path}")
        return model_path
    except Exception as e:
        print(f"> Error saving model: {str(e)}")
        raise

def load_model(architecture_name, data_path, split_index):
    model_name = f"pretext_model_{os.path.basename(data_path)}_{architecture_name}_split_{split_index}.h5"
    model_path = os.path.join(MODEL_DIR, model_name)
    
    try:
        if os.path.exists(model_path):
            model = tf.keras.models.load_model(model_path)
        else:
            model_path = model_path.replace('.h5', '')
            model = tf.keras.models.load_model(model_path)
        print(f"> Model successfully loaded from {model_path}")
        return model
    except Exception as e:
        print(f"> Error loading model: {str(e)}")
        raise

def run_pretext_pipeline(data_path, architecture_name, split_index):
    # set_all_seeds(RANDOM_SEED)
    
    model_name = f"pretext_model_{os.path.basename(data_path)}_{architecture_name}_split_{split_index}.h5"
    model_path = os.path.join(MODEL_DIR, model_name)
    
    if os.path.exists(model_path):
        print(f"> Alredy exists in {model_path}. No need for training.")
        return

    x_train, y_train, _, _ = load_processed_data(data_path)
    x_augmented, y_augmented = preprocess_data(x_train)
    
    model = build_custom_cnn_model(
        input_shape=hyperparameters['input_shape'],
        num_classes=hyperparameters['num_classes'],
        architecture_name=architecture_name,
        fine_tune=hyperparameters['fine_tune']
    )
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=hyperparameters['early_stopping_patience'],
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=hyperparameters['lr_reduction_factor'],
            patience=hyperparameters['lr_reduction_patience']
        )
    ]
    
    '''Not proceeding with making a static validation set as not quite sure about the stratified nature'''
    # validation_size = int(len(x_augmented) * hyperparameters['validation_split'])
    # indices = np.arange(len(x_augmented))
    # np.random.seed(RANDOM_SEED)
    # np.random.shuffle(indices)
    # train_idx = indices[validation_size:]
    # val_idx = indices[:validation_size]
    
    # history = model.fit(
    #     x_augmented[train_idx], y_augmented[train_idx],
    #     validation_data=(x_augmented[val_idx], y_augmented[val_idx]),
    #     batch_size=hyperparameters['batch_size'],
    #     epochs=hyperparameters['epochs'],
    #     callbacks=callbacks,
    #     shuffle=False
    # )

    history = model.fit(
        x_augmented, y_augmented,
        validation_split = hyperparameters['validation_split'],
        batch_size=hyperparameters['batch_size'],
        epochs=hyperparameters['epochs'],
        callbacks=callbacks,
        verbose=2
    )
    
    # Save the trained model
    save_model(model, architecture_name, data_path, split_index)
    
    return history

def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    
    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Accuracy over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [16]:
# [Cell 8] - Downstream Task Functions
def extract_features(pretext_model, x_data, layer_index=-2):
    intermediate_model = models.Model(inputs=pretext_model.input, outputs=pretext_model.layers[layer_index].output)
    features = intermediate_model.predict(x_data, verbose=2)
    print(f'> extracted features of shape {features.shape}')
    return features

def evaluate_downstream_task(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    return {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='macro'),
        'recall': recall_score(y_test, y_pred, average='macro'),
        'f1': f1_score(y_test, y_pred, average='macro'),
        'confusion_matrix': confusion_matrix(y_test, y_pred)
    }

def train_downstream_task(train_features, train_labels, test_features, test_labels, classifier='random_forest', n_splits=None):
    print(f'> Performing downstream task with {classifier}')
    
    # Assign number of splits from hyperparameters if not specified
    if n_splits is None:
        n_splits = hyperparameters['cv_splits']
    
    # Initialize classifier based on hyperparameters
    if classifier == 'random_forest':
        clf = RandomForestClassifier(
            n_estimators=hyperparameters['rf_n_estimators'],
            random_state=RANDOM_SEED,
            verbose=0
        )
    elif classifier == 'svm':
        clf = SVC(
            kernel=hyperparameters['svm_kernel'],
            random_state=RANDOM_SEED,
            verbose=False
        )
    elif classifier == 'gradient_boosting':
        clf = GradientBoostingClassifier(
            n_estimators=hyperparameters['gb_n_estimators'],
            random_state=RANDOM_SEED,
            verbose=0
        )
    elif classifier == 'xgboost':
        clf = XGBClassifier(
            n_estimators=hyperparameters['xgb_n_estimators'],
            random_state=RANDOM_SEED,
            use_label_encoder=False,  # Update for recent versions of XGBoost
            verbose=0
        )
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    cv_scores = {
        'accuracies': [], 'precisions': [], 'recalls': [], 'f1_scores': []
    }
    
    for train_idx, val_idx in skf.split(train_features, train_labels):
        X_train_fold, X_val_fold = train_features[train_idx], train_features[val_idx]
        y_train_fold, y_val_fold = train_labels[train_idx], train_labels[val_idx]
        
        clf.fit(X_train_fold, y_train_fold)
        y_pred = clf.predict(X_val_fold)
        
        # Append cross-validation scores
        cv_scores['accuracies'].append(accuracy_score(y_val_fold, y_pred))
        cv_scores['precisions'].append(precision_score(y_val_fold, y_pred, average='macro'))
        cv_scores['recalls'].append(recall_score(y_val_fold, y_pred, average='macro'))
        cv_scores['f1_scores'].append(f1_score(y_val_fold, y_pred, average='macro'))
    
    # Train final classifier on full training data and evaluate on test data
    clf.fit(train_features, train_labels)
    test_metrics = evaluate_downstream_task(clf, test_features, test_labels)
    
    return clf, cv_scores, test_metrics

def run_downstream_pipeline(data_path, architecture_name, downstream_classifier, split_index):
    x_train, y_train, x_test, y_test = load_processed_data(data_path)
    pretext_model = load_model(architecture_name, data_path, split_index)
    
    train_features = extract_features(pretext_model, x_train, hyperparameters['feature_extraction_layer'])
    test_features = extract_features(pretext_model, x_test, hyperparameters['feature_extraction_layer'])
    
    clf, cv_scores, test_metrics = train_downstream_task(
        train_features, y_train,
        test_features, y_test,
        classifier=downstream_classifier,
        n_splits=hyperparameters['cv_splits']
    )
    
    return clf, cv_scores, test_metrics

In [17]:
# [Cell 10] - Main Execution
def main():
    set_all_seeds(RANDOM_SEED)

    # Check for GPU
    print("> GPU Availability: ", tf.config.list_physical_devices('GPU'))
    
    # Parameters
    mode = experiment_parameters['mode']
    split_index = experiment_parameters['split_index']
    architecture = experiment_parameters['architecture']
    classifier = experiment_parameters['classifier']
    
    # Data directory
    data_dir = experiment_parameters['data_dir']
    
    try:
        # Step 1: Process and save data
        if mode in ['process', 'all']:
            print("\n=== Processing Data ===")
            data_path = save_processed_data(data_dir, split_index)
        
        # Step 2: Train pretext model
        if mode in ['pretext', 'all']:
            print("\n=== Training Pretext Model ===")
            data_path = os.path.join(DATA_DIR, f"{data_dir.split('/')[-2]}_{data_dir.split('/')[-1]}_split_{split_index}")
            
            # TODO, load the data here

            architectures = architecture_list if architecture == 'all' else [architecture]
            for arch in architectures:
                print(f">> Training architecture: {arch}")
                history = run_pretext_pipeline(data_path, arch, split_index)

            # TODO, destroy data memory here
        
        # Step 3: Run downstream task
        if mode in ['downstream', 'all']:
            print("\n=== Running Downstream Task ===")
            data_path = os.path.join(DATA_DIR, f"{data_dir.split('/')[-2]}_{data_dir.split('/')[-1]}_split_{split_index}")
            # TODO, load the data here
            
            results_tracker = ResultsTracker()
            architectures = architecture_list if architecture == 'all' else [architecture]
            classifiers = classifier_list if classifier == 'all' else [classifier]
            
            for arch in architectures:
                # TODO, Load the architecture here
                for clf in classifiers:
                    print(f">> Evaluating classifier: {clf} with architecture: {arch}")
                    classifier_model, cv_scores, test_metrics = run_downstream_pipeline(
                        data_path, arch, clf, split_index
                    )
                    
                    results_tracker.add_result(
                        data_dir=data_dir,
                        architecture=arch,
                        classifier=clf,
                        cv_metrics=cv_scores,
                        test_metrics=test_metrics
                    )
                results_tracker.display_results()
                # TODO, destroy architecture memory here

            # TODO, destroy data memory here

            results_tracker.display_results()

    except Exception as e:
        print(f"\nError occurred: {str(e)}")
        raise

In [18]:
# [Cell 11] - Run the pipeline
if __name__ == "__main__":
    timestamp = datetime.now().strftime('%Y-%m-%d-%Hh%Mm%Ss')
    filename = os.path.join(RESULTS_DIR, f"{timestamp}-dir_{experiment_parameters['data_dir'].split('/')[-2]}_{experiment_parameters['data_dir'].split('/')[-1]}-mode_{experiment_parameters['mode']}-split_{experiment_parameters['split_index']}-arch_{experiment_parameters['architecture']}-clf_{experiment_parameters['classifier']}.txt")
    print(f"> Output is being saved on {filename}")

    try:
        with open(filename, 'w') as output_file:
            sys.stdout = output_file
            display_configurations()
            main()
            sys.stdout = sys.__stdout__
    except Exception as e:
        sys.stdout = sys.__stdout__
        print("ERROR OCCURED", e)
        
    print("Process finished.")

/home/siam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/siam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/siam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/siam/.local/lib/python